In [1]:
require 'rnn'
require 'itorch.Plot'

In [2]:
-- TODO something less hardcoded
filenames = {}
for i=2,9 do
    for _, typ in ipairs({'full', 'lstm', 'rnn', 'fulltwo'}) do
        table.insert(filenames, string.format('models_%d_%s/rubiks_best', i, typ))
    end
end

best_results = {}
for _, name in ipairs(filenames) do
    best_results[name] = torch.load(name, 'ascii')
    print(name, 'loaded')
end

models_2_full/rubiks_best	loaded	


models_2_lstm/rubiks_best	loaded	


models_2_rnn/rubiks_best	loaded	


models_2_fulltwo/rubiks_best	loaded	


models_3_full/rubiks_best	loaded	


...

models_8_fulltwo/rubiks_best	loaded	


models_9_full/rubiks_best	loaded	


models_9_lstm/rubiks_best	loaded	


models_9_rnn/rubiks_best	loaded	


models_9_fulltwo/rubiks_best	loaded	


In [4]:
-- Training accuracy plot
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
full2_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
lstm_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full/rubiks_best', i)
    full_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_fulltwo/rubiks_best', i)
    full2_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_rnn/rubiks_best', i)
    rnn_y[i-1] = best_results[name].train_acc
    name = string.format('models_%d_lstm/rubiks_best', i)
    lstm_y[i-1] = best_results[name].train_acc
end

plot:xaxis('Episode Length'):yaxis('Training Accuracy')
plot:title('Training accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, full2_y, 'pink', 'Fully Connected 2')
plot:line(x, rnn_y, 'blue', 'RNN')
plot:line(x, lstm_y, 'green', 'LSTM')
plot:legend(true)
plot:draw()

In [5]:
-- Test accuracy plot
plot = itorch.Plot()
x = torch.range(2, 9)
full_y = torch.Tensor(8)
rnn_y = torch.Tensor(8)
lstm_y = torch.Tensor(8)
for i=2,9 do
    name = string.format('models_%d_full/rubiks_best', i)
    full_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_fulltwo/rubiks_best', i)
    full2_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_rnn/rubiks_best', i)
    rnn_y[i-1] = best_results[name].test_acc
    name = string.format('models_%d_lstm/rubiks_best', i)
    lstm_y[i-1] = best_results[name].test_acc
end

plot:xaxis('Episode Length'):yaxis('Test Accuracy')
plot:title('Test accuracy (50000 episodes)')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, full2_y, 'pink', 'Fully Connected 2')
plot:line(x, rnn_y, 'blue', 'RNN')
plot:line(x, lstm_y, 'green', 'LSTM')
plot:legend(true)
plot:draw()

In [6]:
require 'csvigo'
data = csvigo.load('test_results/scores.csv')
-- Divide into the ones for each model type
-- This assumes the model names are sorted by episode length, then by type
-- in alphabetical order
percents = torch.Tensor(data.percent)
inds = torch.range(1, 4 * 8, 4):type('torch.LongTensor')

x = torch.range(2,9)
full_y = percents:index(1, inds)
full2_y = percents:index(1, inds + 1)
lstm_y = percents:index(1, inds + 2)
rnn_y = percents:index(1, inds + 3)

-- TODO describe this graph more
-- for episode length i, tested on length i + 1.
plot = itorch.Plot()
plot:xaxis('Episode Length Trained On'):yaxis('Solve Percentage')
plot:title('Solve Percentage in Real World Tests')
plot:line(x, full_y, 'red', 'Fully Connected')
plot:line(x, full2_y, 'pink', 'Fully Connected 2')
plot:line(x, rnn_y, 'blue', 'RNN')
plot:line(x, lstm_y, 'green', 'LSTM')
plot:legend(true)
plot:draw()

<csv>	parsing file: test_results/scores.csv	
<csv>	tidying up entries	
<csv>	returning tidy table	


In [7]:
require 'csvigo'
data = csvigo.load('models_26_lstm_tests/scores.csv')
percents = torch.Tensor(data.percent)
x = torch.range(1, 26)

plot = itorch.Plot()
plot:xaxis('Scramble Length'):yaxis('Solve Percentage')
plot:title('Solve Percentages On Random Scrambles')
plot:line(x, percents, 'red', 'LSTM (26)')
plot:legend(true)
plot:draw()

<csv>	parsing file: models_26_lstm_tests/scores.csv	
<csv>	tidying up entries	
<csv>	returning tidy table	


In [28]:
-- Comparing models with the same structure trained on different scramble lengths
require 'csvigo'
-- I know 'data' is already plural. Whatever
datas = {}
percs = {}
for i = 2,9 do
    datas[i] = csvigo.load(string.format('models_%d_lstm_tests/scores.csv', i))
    percs[i] = torch.Tensor(datas[i].percent)
end
datas[10] = csvigo.load('models_26_lstm_tests/scores.csv', i)
percs[10] = torch.Tensor(datas[10].percent)

x = torch.range(1, 26)

plot = itorch.Plot()
plot:xaxis('Scramble Length'):yaxis('Solve Percentage')
plot:title('Solve Percentages On Random Scrambles')
colors = {
    'red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'gray', 'black'
}
for i = 2,9 do
    plot:line(x, percs[i], colors[i-1], string.format('LSTM (%d)', i))
end
plot:line(x, percs[10], colors[9], 'LSTM (26)')
plot:legend(true)
plot:draw()

<csv>	parsing file: models_2_lstm_tests/scores.csv	


<csv>	tidying up entries	


<csv>	returning tidy table	
<csv>	parsing file: models_3_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_4_lstm_tests/scores.csv	
<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_5_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_6_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_7_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_8_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_9_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_26_lstm_tests/scores.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	


In [1]:
-- General learning curve data loader
require 'lfs'
function find_names(dir)
    -- Finds all output files. Returns (table of names, max epoch)
    iter, dir_obj = lfs.dir(dir)
    pattern = 'rubiks_epoch(%d+)'
    name = iter(dir_obj)
    names = {}
    max = -1
    while name ~= nil do
        num = string.match(name, pattern)
        if num ~= nil then
            num = tonumber(num)
            names[num] = dir .. '/' .. name
            if num > max then
                max = num
            end
        end
        name = iter(dir_obj)
    end
    return names, max
end

In [2]:
function data_loader(names, epoch_max, attr_names)
    -- Loads data from each filename given, fetches the attribute names, and
    -- puts it into a Tensor
    local data = {}
    for i = 1, #attr_names do
        data[attr_names[i]] = torch.Tensor(epoch_max)
    end
    for i = 1, epoch_max do
        local dat = torch.load(names[i], 'ascii')
        for j = 1, #attr_names do
           data[attr_names[j]][i] = dat[attr_names[j]] 
        end
    end
    return data
end

In [ ]:
-- PRETTY SLOW
require 'rnn'

lstm_data = {}
for i = 5, 9 do
    lstm_data[i] = {}
    names, max = find_names('models_more_data/' .. i .. '_lstm')
    lstm_data[i] = data_loader(names, max, {'train_acc', 'train_err', 'test_acc', 'test_err'})
    print('finished ' .. i)
end
--save so I don't have to do that junk again
torch.save('lstm_more_data', lstm_data, 'ascii')

In [26]:
plot = itorch.Plot()
x = torch.range(1,39)
y = lstm_data[9].train_acc
plot:line(x, y, 'red', 'Train Acc')
y2 = lstm_data[9].test_acc
plot:line(x, y2, 'blue', 'Test Acc')
plot:legend(true)
plot:draw()

In [30]:
-- Comparing models with the same structure trained on different scramble lengths
require 'csvigo'
-- I know 'data' is already plural. Whatever
datas = {}
percs = {}
datas[1] = csvigo.load(string.format('models_more_data/9_lstm_tests/scores.csv', i))
percs[1] = torch.Tensor(datas[1].percent)
datas[2] = csvigo.load(string.format('models_9_lstm_tests/scores.csv'))
percs[2] = torch.Tensor(datas[2].percent)

x = torch.range(1, 26)

plot = itorch.Plot()
plot:xaxis('Scramble Length'):yaxis('Solve Percentage')
plot:title('Solve Percentages On Random Scrambles')
plot:line(x, percs[1], 'red', 'LSTM (9) (10^5 eps)')
plot:line(x, percs[2], 'blue', 'LSTM (9) (50000 eps)')
plot:legend(true)
plot:draw()

<csv>	parsing file: models_more_data/9_lstm_tests/scores.csv	
<csv>	tidying up entries	
<csv>	returning tidy table	
<csv>	parsing file: models_9_lstm_tests/scores.csv	
<csv>	tidying up entries	
<csv>	returning tidy table	
